# Planejamento do projeto

## Resultado final
    - Uma aplicação em que o usuário procura um determinado ativo (ação ou fundo) e receber um gráfico com o hostórico e os principais indicadores fundamentalistas.
    - Quais análises serão entregues?
        . Gráfico interativo com o comportamento das ações no tempo;
        . Relatório com os principais indicadores;
       

## Ferramental
    - Utilizar o Jupyter Notebook para a escrita e interpretação do código;
    - Utilizar a biblioteca Selenium e/ou urllib.request para analisar conteúdos de páginas;
    - Utilizar a biblioteca Beautiful Soup (ou lXml e Scrapy) para extrair o conteúdo das páginas HTML;
    - Utilizar a biblioteca Pandas para manipular os dados das páginas em DataFrames;
    - Utilizar a biblioteca Numpy para realização de operações;
    - Utilizar a biblioteca Plotly para desenhar gráficos interativos;
    - Utilizar Streamlit para a criação de dahboard interativo e aplicação;
    - Utilização de Spyder ou outro interpretador de python para a geração da aplicação.
    
## Processo de desenvolvimento
    - Coletar os dados das páginas que contêm informações sobre ações: 
        - https://finance.yahoo.com/ 
        - https://statusinvest.com.br/
        - https://www.fundsexplorer.com.br/
            - Dados históricos dos preços das ações;
            - Indicadores fundamentalistas das ações; 
            - Dados históricos dos fundos;
            - Indicadores dos fundos;
    - Organizar os dados em DataFrames;
    - Tratar os dados, isto é, retirar dados duplicados, dados faltantes, ajustar datas, índices e headers;
    - Gerar as análises dos dados;
    - Definir do layout da aplicação
    - Codificar o layout e análises geradas no Jupyter

# Raspagem da tabela de preços de ações utilizando o Selenium

In [1]:
# Imports para o desenvolvimento do projeto
import time
import requests
import csv
import pandas as pd 
import numpy as np
# import urllib.request as ur
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service


In [ ]:
# Função para escrever o arquivo csv
def write_csv_file(file_path, content):
    try:
        with open(file_path, mode='w') as csv_file:
            # Cria o ponteiro para a escrita do arquivo csv
            writer = csv.writer(csv_file)

            # Escreve todas a linhas de uma só vez sem necessidade de loop
            writer.writerows(content)
        
        return True
    except Exception as event:
        print(event)
        return False

In [ ]:
# Ler o arquivo csv e fazer a lista de opções para o filtro
def read_csv_file(file_path): #apenas testando função
    output_list = []
    
    with open(file_path, mode='r', newline='\n') as csv_file:
        # Cria o ponteiro para a escrita do arquivo
        reader = csv.reader(csv_file)
        
        # Transfere as linhas para a lista
        for line in reader:
            output_list.append(line)
    
    return output_list

In [2]:
# Inserir o nome empresa ou fundo
ticker = input()

SULA11.SA


In [6]:
# Ajustano o URL
# selecionando a data do horizonte de busca de 5 anos 
date2 = np.timedelta64(np.datetime64('today') - np.datetime64('1969-12-31'), 's').astype(int)
date1 = date2 - 86400 * 365 * 5

url = "https://br.financas.yahoo.com/quote/{}/history?period1={}&period2={}&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"

# inserindo elementos no URL da página que será acessada
url = url.format(ticker, date1, date2)

# Instância do navegador
opt = webdriver.ChromeOptions()
opt.headless = True #não mostrar a ação em andamento 
ser = Service(r'browser\chromedriver.exe') # caminho para o driver
driver = webdriver.Chrome(service=ser, options=opt)

driver.get(url) #faz o drive do navegador acessar o URL 

time.sleep(5) #tempo de pausa para os processos paralelos

html = driver.find_element(By.TAG_NAME, 'html') #localiza o início da página 

# posição do inicial do scroll
last_height = driver.execute_script("return document.body.scrollHeight")
print("last_height: {}".format(last_height))

while (True):
    # posiciona o scroll no final da página
    html.send_keys(Keys.END)
    
    # pausa para carregar a página
    time.sleep(1)

    # atualiza a posição do scroll 
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    print("new_height: {}".format(new_height))
    
    # verifica se houve movimento da página 
    if (new_height == last_height):
        # termina o loop
        break
    else:
        # atualiza o último valor 
        last_height = new_height         

element = driver.find_element(By.TAG_NAME,'table')
html_content = element.get_attribute('outerHTML')

driver.quit() #fecha o navegador

# Código HTML da tabela
soup = BeautifulSoup(html_content, 'html.parser')

# Tratando o HTML para gerar a tabela de hostórico do preço da ação
# lista para armazenar o dados que não sejam relacionados com o preço da ação (dividendo e desdobramento)
list_not_related = [] #lista com conteúdo não relacionado com o preço das ações
tuplas_eventos = [] #tuplas data, valor, ocorrencia (dividendo e desdobramento)
for element in soup.find_all('td',"Ta(start) Py(10px)"):
    list_not_related.append(element)
    tuplas_eventos.append((element.previous_element, element.find('strong').string, element.find('span').string))

linhas_tab = [] #lista para armazenar os dados das linhas da tebela
# retira os strings que acompanham as classes selecionadas e filtra os casos não desejados
for element in soup.find_all(['td', 'th']):
    if element in list_not_related:
        del(linhas_tab[-1]) #deleta o elemento anterior
    else:
        linhas_tab.append(element.string)
        
new_lista = list(filter(None, linhas_tab)) #retira os Nones se existirem
stock_data = list(zip(*[iter(new_lista)]*7)) #empacota em listas de 7 elementos 

# Cria um dataframe com os valores não desejados
df_data_stocks = pd.DataFrame(stock_data[1:], columns=stock_data[0][0:7]) 
df_not_related = pd.DataFrame(tuplas_eventos, columns=['Data', 'Valor', 'Tipo'])


# Tratando o DataFrame
# converte a última coluna para inteiro substituindo o ponto
df_data_stocks["Volume"] = df_data_stocks['Volume'].apply(lambda x: int(x.replace('.','').replace(".","")) if x != "-" else x)

# laço para substituir os pontos por vírgulas 
for coluna in df_data_stocks.columns[1:6]:
    df_data_stocks[coluna] = df_data_stocks[coluna].apply(lambda x: float(x.replace('.','').replace(",",".")) if x != '-' else x)

# dicionário para auxiliar na correção das datas
dicio = {"jan.": "01",
         "fev.": "02",
         "mar.": "03",
         "abr.": "04",
         "mai.": "05",
         "jun.": "06",
         "jul.": "07",
         "ago.": "08",
         "set.": "09",
         "out.": "10",
         "nov.": "11",
         "dez.": "12"}

# ordenando cada data por Ano-mês-dia, substituindo o nome do mês pelo número correspondente e convertendo para datetime64
df_data_stocks["Data_Time"] = pd.to_datetime(df_data_stocks["Data"].apply(lambda x: x.split(' ')[0:6:2][::-1]).\
                                             apply(lambda x: "".join([x[0], dicio[x[1]], x[2]])), 
                                             format='%Y-%m-%d')

# Rearanjando as colunas do dataframe
cols = df_data_stocks.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_data_stocks = df_data_stocks[cols]
df_data_stocks.index = df_data_stocks['Data_Time'] #faz o índice virar a coluna data_time
df_data_stocks.drop(labels='Data_Time', axis=1, inplace=True)

df_data_stocks

last_height: 583
new_height: 8303
new_height: 12092
new_height: 15881
new_height: 19670
new_height: 23459
new_height: 27248
new_height: 31037
new_height: 34826
new_height: 38615
new_height: 42404
new_height: 46193
new_height: 48629
new_height: 48629


,Data,Abrir,Alto,Baixo,Fechamento*,Fechamento ajustado**,Volume
Data_Time,,,,,,,
2022-03-18,18 de mar. de 2022,33.67,34.63,33.62,34.51,34.51,3990300
2022-03-18,18 de mar. de 2022,33.67,34.63,33.62,34.35,34.35,2400800
2022-03-17,17 de mar. de 2022,34.25,34.25,33.50,33.85,33.85,3279800
2022-03-16,16 de mar. de 2022,33.60,34.26,33.11,34.25,34.25,4025200
2022-03-15,15 de mar. de 2022,33.22,33.65,32.63,33.46,33.46,4665500
...,...,...,...,...,...,...,...
2017-03-27,27 de mar. de 2017,15.88,15.92,15.60,15.87,13.54,715124
2017-03-24,24 de mar. de 2017,15.78,16.09,15.73,15.93,13.60,663133
2017-03-23,23 de mar. de 2017,15.56,16.03,15.47,15.80,13.49,954707


# Gráfico interativos

In [ ]:
# Imports para a produção do gráfico interativo
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import fixed
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# O filtro SelectionRangeSlider precisa de uma lista de valores para exibir para o usuário
options = [e.strftime(' %d/%m/%Y ') for e in df_data_stocks.index.to_list()[::-1]]

# Cria o filtro SelectionRange para a data
date_filter = widgets.SelectionRangeSlider(
    options=options,
    ensure_option=True,
    index=(0, len(options)-1), #fornecer os índices da lista
    description='Filtro Data:',
    disabled = False,
    layout={'width': '500px'},
    continuous_update=False
)

# Opções para o combobox
options = df_data_stocks.columns.to_list()[1::]

# Criando a seleção de colunas do DataFrame para exibir no gráfico
column_filter = widgets.Combobox(
    value='Fechamento*',
    placeholder='Clique para escolher',
    options=options,
    description='Parâmetro de análise:',
    ensure_option=True,
    style={'description_width':'initial'}, #mostrar a "description" completa
    layout={'width': '350px'},
    disabled = False
)

def update_graph(df, filter_date, filter_column):
    # Datas selecionadas, convertidas de string (%d/%m/%Y) para datetime64 (%Y-%m-%d)
    date_init = np.datetime64("-".join(filter_date[0].replace('/','-').strip().split('-')[::-1])) 
    date_fin = np.datetime64("-".join(filter_date[1].replace('/','-').strip().split('-')[::-1]))
    
    # Fatiando o dataframe conforme filtro de data
    df_test = df[(df.index >= date_init) &
                 (df.index <= date_fin)].copy()
    
    # Plotando o gráfico conforme data selecionada e coluna
    fig = go.Figure([go.Scatter(x=df_test.index, y=df_test[filter_column])])
    fig.update_xaxes(dtick="M20", tickformat="%b \n%Y")
    fig.show()
    
    return None
    
    
widgets.interactive(update_graph, df=fixed(df_data_stocks), filter_date=date_filter, filter_column=column_filter)

# Tabela de indicadores de FII

In [ ]:
url = 'https://www.fundsexplorer.com.br/ranking'
header ={'user-agent':'Mozilla/5.0'}
read_url = requests.get(url, headers = header)

# Tratando o conteúdo do site
soup = BeautifulSoup(read_url.content,'html.parser')

lits_td = [] #lista com objetos da classe td
for i in soup.find_all('td'):
    lits_td.append(i.string) 

index = 0
list_funds = [] #lista com as infos finais dos fundos
while index < len(lits_td):
    temp_lits = []
    for z in range(index, index + 25):
        temp_lits.append(lits_td[z])
    list_funds.append(temp_lits)
    index += 26

columns = ['Código do fundo', 'Setor', 
           'Preço atual', 'Liquidez',
           'Dividendo', 'Dividend Yield',
           'DY 3M', 'DY 6M', 'DY 12M', 
           'DY 3M media', 'DY 6M media',
           'DY 12M media', 'DY ANO', 
           'Variação Preço', 'Rentab. Período',
           'Retab. Acum.', 'Patrimônio Líq.',
           'VPA', 'P/VPA', 'DY PATR.', 
           'Varia. Patri.', 'Rentab. Patr.', 
           'Vacância Física', 'Vacância Financeira',
           'Quantidade de ativos']

df_fundos = pd.DataFrame(list_funds, columns=columns)
df_fundos.index = df_fundos["Código do fundo"]
df_fundos.drop(labels='Código do fundo', axis=1, inplace=True)

In [ ]:
# Salvando lista com nomes dos fundos setores
# lista combinando os nomes dos fundo e o setor deles
result_list = list(zip(df_fundos.index.tolist(), df_fundos['Setor'].tolist()))
file_name = "fundos_listados.csv" #nome do arquivo de saída

# Escrevendo em um arquivo csv
write_csv_file(file_name, result_list)

In [ ]:
# Separando os indicadores desejados 
fund_ticker = 'XPLG11'
indices_fundos = {'FII': fund_ticker}
indices_fundos['Preço'] = df_fundos['Preço atual'][fund_ticker]
indices_fundos['DY 12 M'] = df_fundos['DY 12M'][fund_ticker]
indices_fundos['Vacância Financeira'] = df_fundos['Vacância Financeira'][fund_ticker]
indices_fundos['P/VPA'] = df_fundos['P/VPA'][fund_ticker] 
indices_fundos['Dividendo'] = df_fundos['Dividendo'][fund_ticker]
indices_fundos['Magic Number'] = str(round(float(df_fundos['Preço atual'][fund_ticker].replace('R$ ','').replace(',','.')) / 
                                     float(df_fundos['Dividendo'][fund_ticker].replace('R$ ','').replace(',','.')), 2)).replace('.',',')

indices_fundos

# Indicadores fundamentalistas para ações

In [ ]:
# Imports necessários
import pandas as pd 
import requests
from bs4 import BeautifulSoup

In [ ]:
Input_ticker = 'SULA11'

# Aquisição do HTML
url = 'https://statusinvest.com.br/acoes/{}'
url = url.format(Input_ticker)

headers = {'User-Agent':'Mozilla/5.0'}
url_requests = requests.get(url, headers=headers)
soup = BeautifulSoup(url_requests.text, 'html.parser')

# Localização dos dados alvo
list_target = []
for i in soup.find_all('div','info special w-100 w-md-33 w-lg-20'):
    list_target.append((i.find('h3').string, i.find('strong').string))
        
for i in soup.find_all('div',{'title':'Valorização no preço do ativo com base nos últimos 12 meses'}):
    list_target.append((i.find('h3').string, i.find('strong').string))

for i in soup.find_all('div',["w-50 w-sm-33 w-md-25 w-lg-50 mb-2 mt-2 item",'w-50 w-sm-33 w-md-25 w-lg-16_6 mb-2 mt-2 item']):
    list_target.append((i.find('h3').string, i.find('strong').string))

df_indices_statusinvest = pd.DataFrame(list_target, columns=['Indicador','Valor'])

In [ ]:
# Separando os indicadores desejados 
indices_acoes = {'Acao': Input_ticker}
indices_acoes['ROE'] = df_indices_statusinvest.loc[df_indices_statusinvest['Indicador'].isin(['ROE']), "Valor"].item()
indices_acoes['ROIC'] = df_indices_statusinvest.loc[df_indices_statusinvest['Indicador'].isin(['ROIC']), "Valor"].item()
indices_acoes['Liq. corrente'] = df_indices_statusinvest.loc[df_indices_statusinvest['Indicador'].isin(['Liq. corrente']), "Valor"].item()
indices_acoes['M. Líquida'] = df_indices_statusinvest.loc[df_indices_statusinvest['Indicador'].isin(['M. Líquida']), "Valor"].item()
indices_acoes['CAGR Receitas 5 anos'] = df_indices_statusinvest.loc[df_indices_statusinvest['Indicador'].isin(['CAGR Receitas 5 anos']), "Valor"].item()
indices_acoes['P/L'] = df_indices_statusinvest.loc[df_indices_statusinvest['Indicador'].isin(['P/L']), "Valor"].item()

indices_acoes

# Obtendo a lista de empresas listadas na bolsa

In [ ]:
url = "https://www.infomoney.com.br/cotacoes/empresas-b3/"
headers = {'user-agent':'Mozilla/5.0'}
url_content = requests.get(url, headers)
soup = BeautifulSoup(url_content.content, 'html.parser')

firms = []
# Varre o código html para extrair informações desejadas
for each_tr in soup.find_all("tr"):
    # Extrai o nome da empresa
    firm_name = each_tr.find('td', 'higher').string.strip() if each_tr.find('td', 'higher') != None else None
    
    tckrs_list = []
    # Extrai as variações de ticker
    for each_a in each_tr.find_all('a'):
        tckrs_list.append(each_a.string)
    
    # Relacionando cada ticker a sua empresa
    for tckr in tckrs_list:
        firms.append([firm_name, tckr])

firms = list(filter(lambda x: True if None not in x else False, firms)) #retira os Nones
firms.sort()

In [ ]:
# Escrevendo a lista de saída em um arquivo csv
file_name = 'data\\csvs\\empresas_listadas.csv'

write_csv_file(file_name, firms):

In [ ]:
# Lendo o csv das empresas listadas
options = tuple([element[1] + " (" + element[0] + ")" for element in read_csv_file("data\\csvs\\empresas_listadas.csv")])

In [ ]:
# Exemplo de retirada de Nones utilização do filter
def check_None(teste): #apenas teste
    for e in teste:
        if e != None:
            for a in e:
                if a != None:
                    return True
            else:
                return True
        return False

new_lista = list(filter(check_None, teste)) #retira os Nones